<a href="https://colab.research.google.com/github/jonasvdbran/Masterthesis-SCRPPP/blob/main/Container_classes_and_generating_modules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random

# Classes

**Containers**

In [ ]:
class Container:
  def __init__(self,value,ID="",placed=False):
    # A container has a value, representing it's dwell time.
    # A container also has a binary 'placed' attribute, this is altered in the pre-processing algorithm,
    # when a container has been processed.
    self.value=value
    self.placed=placed
    self.ID=ID

  # 2 different ways to display a container
  def __str__(self):
    return str(self.value)
  def __repr__(self):
    return f"Container{self.value,self.ID,self.placed}"

  def copy(self):
    return Container(self.value,self.ID,self.placed)

  # Comparison of containers is based on it's value, can't compare containers with other objects
  def __le__(self, other):
    if isinstance(other,Container):
      return self.value<=other.value
    else:
      raise TypeError("Can't compare instances of class Container with other objects")

  def __ge__(self, other):
    if isinstance(other,Container):
      return self.value>=other.value
    else:
      raise TypeError("Can't compare instances of class Container with other objects")

  def __lt__(self, other):
    if isinstance(other,Container):
      return self.value<other.value
    else:
      raise TypeError("Can't compare instances of class Container with other objects")

  def __gt__(self, other):
    if isinstance(other,Container):
      return self.value>other.value
    else:
      raise TypeError("Can't compare instances of class Container with other objects")
  def __eq__(self,other):
    if isinstance(other,Container):
      return self.value==other.value
    else:
      raise TypeError("Can't compare instances of class Container with other objects")
  def __ne__(self, other):
    if isinstance(other, Container):
        return self.value != other.value
    return TypeError("Can't compare instances of class Container with other objects")

**Stacks**

In [ ]:
class Stack:
  def __init__(self,lst,max_height,ID=""):
    # A stack consists of a list of container type objects
    # Each stack has a max_height, its length cannot exceed this.
    # The first container of the list is the bottom container, the last is the top
    self.ID=ID
    if max_height <= 0:
        raise ValueError("Max height must be positive.")

    if len(lst)>max_height:
      raise ValueError("Initial list exceeds the maximum allowed length.")
    if not all(isinstance(container, Container) for container in lst):
        raise TypeError("All items in the stack must be instances of the Container class.")

    self.max_height=max_height
    self.containers=list(lst)

  def __len__(self):
    return len(self.containers)

  def __getitem__(self, index):
    if isinstance(index, slice):
        container_values= [container.value for container in self.containers]
        return container_values[index]

    elif isinstance(index, (int, np.integer)):
        if index >= len(self.containers):
            raise IndexError("Stack index out of range.")
        else:
          return self.containers[index]
    else:
        raise TypeError("Invalid argument type. Index must be an integer or a slice.")


  def copy(self):
    return Stack([container.copy() for container in self.containers],self.max_height,self.ID)

  # We can ask a stack if it's full or empty
  def is_full(self):
    return len(self.containers)==self.max_height
  def is_empty(self):
    return len(self.containers)==0

  # We can push a container on top of the stack or pop the top container.
  def pop(self):
    if self.is_empty():
      raise IndexError("Cannot pop item: the stack is empty.")
    # The top container of a stack can be popped (the top being the last one in the list)
    container= self.containers.pop(-1)
    return container

  def push(self,container):
    # A container cannot be pushed on the stack if it's full
    if self.is_full():
      raise OverflowError("Cannot add item: the stack is full.")
    self.containers.append(container)

  # 2 ways of displaying a stack
  def __str__(self):
    return "Stack: \n"+"\n".join(str(item) for item in reversed(self.containers))
  def __repr__(self):
    return f"Stack({self.containers})"

  # We can iterate over a stack
  def __iter__(self):
      return iter(self.containers)

**Bays**

In [ ]:
class Bay:
  def __init__(self,lst_stacks,max_height,ID=""):
    # A bay consists of a list of stacks. Each stack must have the same max_height. A bay is a 2D object (# stacks, max_height).
    if any(stack.max_height!=max_height for stack in lst_stacks):
      raise ValueError("All stacks must have the same max height.")

    if not all(isinstance(stack, Stack) for stack in lst_stacks):
        raise TypeError("All items in the bay must be instances of the Stack class.")

    self.stacks=lst_stacks
    self.max_height=max_height
    self.containers=[container for stack in self.stacks for container in stack.containers]
    self.ID=ID

  def size(self):
    return (len(self.stacks),self.max_height)

  def copy(self):
    return Bay([stack.copy() for stack in self.stacks],self.max_height,self.ID)

  def __getitem__(self, index):
        # Handles the tuple indexing case, returns container
        if isinstance(index,tuple):
          if len(index)>2:
            raise IndexError("Index has too many dimensions")
          else:
            return self.stacks[index[0]][index[1]]

        # Handles the integer index case, returns stack
        if isinstance(index,int):
          if index >= len(self.stacks):
              raise IndexError("Stackindex out of range.")
          else:
            return self.stacks[index]
        # Handles the slicing case
        elif isinstance(index,slice):
          return [stack for stack in self.stacks[index]]
        else:
         raise TypeError("Invalid argument type. Index must be an integer, tuple or a slice.")

  def __len__(self):
    return len(self.stacks)

  # We can move top containers from one stack in the bay to another in the bay
  def move(self,stack_index,destination_stack_index,in_place=False):
    if in_place:
      container_bay=self
    else:
      container_bay=self.copy()

    if container_bay.stacks[stack_index].is_empty():
        raise IndexError("Source stack is empty.")
    container=container_bay.stacks[stack_index].pop()
    container_bay.stacks[destination_stack_index].push(container)
    if not in_place:
      return container_bay

  # Find the container with the maximum value across all stacks
  def max_container(self):
    return max(
        (container for container in self.containers if container.value != float('inf')),
        default=Container(-1))  # In case all values are infinity

  def min_container(self):
    return min(
        (container for container in self.containers if container.value != float('inf')),
        default=Container(np.inf))  # In case all values are infinity


  # A bay can find where the containers with specified values are located.
  # If a container is already placed, it won't show up in the result.
  # This is used in the pre_processing
  def where_value(self, value):
    occurrences = []
    for i, stack in enumerate(self.stacks):
        for j, container in enumerate(stack.containers):
            if container.value == value and not container.placed:
                occurrences.append((i, j))
    return occurrences

  # 2 different ways to display a bay
  def __str__(self):
    rows = []
    col_width = max(len(str(container.value)) for stack in self.stacks for container in stack.containers)  # Find the longest number for alignment

    for i in range(self.max_height - 1, -1, -1):
        row = [str(stack.containers[i].value).rjust(col_width) if i < len(stack) else " " * col_width
               for stack in self.stacks]
        rows.append(" | ".join(row))  # Join columns with spacing
    header=[f"Bay {self.ID}:\n"] if self.ID else []
    return "\n".join(header+rows)
  def __repr__(self):
    return f"Bay({self.stacks})"

  def __iter__(self):
    return iter(self.stacks)

**Modules**

In [ ]:
class Module:
  def __init__(self,lst_bays,max_height,ID=""):
    # A module consists of a list of bays. A bay is a 3D object (#bays, #stacks, max_height)
    if any(bay.max_height!=max_height for bay in lst_bays):
      raise ValueError("All stacks must have the same height.")
    if not all(isinstance(bay,Bay) for bay in lst_bays):
      raise TypeError("All items in the module must be instances of the Bay class.")

    self.bays=lst_bays
    self.max_height=max_height
    self.containers=[bay.containers for bay in lst_bays]
    self.stacks=[stack for bay in self.bays for stack in bay]
    self.ID=ID
  def __len__(self):
    return len(self.bays)

  def size(self):
    twoD_size=self.bays[0].size()
    return (len(self.bays),twoD_size[0],twoD_size[1])

  def __iter__(self):
    return iter(self.bays)

  def copy(self):
    return Module([bay.copy() for bay in self.bays],self.max_height,self.ID)

  def __getitem__(self, index):
    # Handles index of type tuple
    if isinstance(index, tuple):
      if len(index) == 3:
      # 3D index case: Access specific container in a stack within a bay
        return self.bays[index[0]][index[1]][index[2]]
      elif len(index) == 2:
      # 2D index case: Access a stack within a bay
        return self.bays[index[0]][index[1]]
      else:
        raise IndexError("Index must be either 2D or 3D (tuple of length 2 or 3).")

    # Handles index of type integer
    elif isinstance(index, int):
      if index >= len(self.bays):
        raise IndexError("Index out of range for bays.")
      else:
        return self.bays[index]

    # Handles index of type slice
    elif isinstance(index, slice):
      return [bay for bay in self.bays[index]]
    else:
        raise TypeError("Invalid argument type. Index must be an integer or a tuple.")
  # We can move top containers from one stack in the module to another stack in the module
  def move(self,stack_index,destination_stack_index,in_place=True):
    if not in_place:
      container_module=self.copy()
    else:
      container_module=self
    if container_module[stack_index[0]][stack_index[1]].is_empty():
        raise IndexError("Source stack is empty.")
    container=container_module.bays[stack_index[0]][stack_index[1]].pop()
    container_module.bays[destination_stack_index[0]][destination_stack_index[1]].push(container)
    if not in_place:
      return container_module

  # Find the container with the maximum value across all stacks
  def max_container(self):
    return max(
        [bay.max_container() for bay in self.bays])

  def min_container(self):
    return min(
        [bay.min_container() for bay in self.bays])

  # 2 different ways to display modules
  def __str__(self):
    return "\n\n".join([f"Bay {i}:\n{str(bay)}" if not bay.ID else f"{str(bay)}" for i, bay in enumerate(self.bays)])


  def __repr__(self):
    return f"Module({repr(self.bays)})"

  # A bay can find where the containers with specified values are located.
  # If a container is already placed, it won't show up in the result.
  # This is used in the pre_processing
  def where_value(self, value):
    occurrences = []
    for i, bay in enumerate(self.bays):
        for j, stack in enumerate(bay.stacks):
          for k, container in enumerate(stack.containers):
            if container.value == value and not container.placed:
                occurrences.append((i,j,k))
    return occurrences

# Generating a module


In [ ]:
def discrete_triangle_distribution(mode,left,right)->int:
  """

  Args:
    mode: mode of distribution
    left: left bound of distribution (included)
    right: right bound of distribution (included)

  Returns: Discretized sample (floor) from triangle distribution in range [left,right], with specified mode.

  """
  return int(round(np.random.triangular(left, mode, right)))

In [ ]:
def generate_stack(max_height:int,max_dwell_time:int,mode:int)-> Stack:
  """

  Args:
    max_height: maximum height of stack
    max_dwell_time: maximum value of containers
    mode: mode of distribution of the height of the stack

  Returns: returns randomly filled stack

  """
  # Gets a random value of height of stack and returns a filled stack with this height.
  height=discrete_triangle_distribution(mode,0,max_height) #Draws a number from 1 to max_height+1 according to triangle distribution
  return Stack([Container(value, False) for value in np.random.randint(1,max_dwell_time+1,size=height)],max_height)

In [ ]:
def generate_bay(stacks,max_height:int,max_dwell_time:int,mode:int):
  """

  Args:
    max_height: maximum height of stacks
    stacks: number of stacks
    max_dwell_time: maximum value of containers
    mode: mode of distribution of the height of the stacks

  Returns: randomly filled bay

  """
  bay= []
  for i in range(stacks):
    bay.append(generate_stack(max_height,max_dwell_time,mode))
  return Bay(bay,max_height)

In [ ]:
def generate_module(bays:int,stacks:int,max_height:int,max_dwell_time:int,mode:int)-> Module:
  """

  Args:
    bays: number of bays
    stacks: number of stacks per bay
    max_height: maximum height of stacks
    max_dwell_time: maximum value of containers
    mode: mode of distribution of the height of the stack

  Returns: randomly filled module

  """
  module=[]
  for i in range(bays):
    bay=generate_bay(stacks,max_height,max_dwell_time,mode)
    module.append(bay)
  return Module(module,max_height)

In [ ]:
#module1=generate_module(5,5,5,6,5)
#print(module1)

# Obtaining a module from a list of stacks


In [ ]:
module_lst=[["1_1_9",169492,0,169493,0,154830,206,156893,319,157753,216,0,0],["1_2_2",147240,116,152115,313,154739,115,156985,112,0,0,0,0],["1_2_3",147340,108,152509,113,154766,207,157005,212,0,0,0,0],["1_2_4",147357,15,152558,212,154800,117,155822,13,0,0,0,0],["1_2_5",148764,17,152246,210,155175,116,157025,812,157689,406,0,0],["1_2_6",148896,106,152597,15,154850,816,156283,14,0,0,0,0],["1_2_7",148997,102,152627,413,154912,207,157013,319,0,0,0,0],["1_2_8",149876,210,152571,213,155187,307,157034,414,157708,322,0,0],["1_2_9",169494,0,169495,0,154952,215,155828,113,0,0,0,0],["1_3_2",147240,116,152115,313,154739,115,156985,112,0,0,0,0],["1_3_3",147340,108,152509,113,154766,207,157005,212,0,0,0,0],["1_3_4",147357,15,152558,212,154800,117,155822,13,0,0,0,0],["1_3_6",148896,106,152597,15,154850,816,156283,14,0,0,0,0],["1_3_7",148997,102,152627,413,154912,207,157013,319,0,0,0,0],["1_3_9",169494,0,169495,0,154952,215,155828,113,0,0,0,0],["1_4_2",150274,121,153210,107,154996,315,157077,412,0,0,0,0],["1_4_3",150340,314,153258,313,155016,10,157098,208,0,0,0,0],["1_4_5",150622,17,153421,216,155263,710,156418,213,0,0,0,0],["1_4_6",150871,18,153331,214,155038,113,157154,707,0,0,0,0],["1_4_7",150924,211,153361,112,155064,207,157181,312,0,0,0,0],["1_4_8",149986,15,152953,8,155319,119,157058,118,0,0,0,0],['1_4_9',169496,0,169497,0,155100,414,156421,12,0,0,0,0],['1_5_2',150274,121,153210,107,154996,315,157077,412,0,0,0,0],['1_5_3',150340,314,153258,313,155016,10,157098,208,0,0,0,0],['1_5_5',150622,17,153421,216,155263,710,156418,213,0,0,0,0],['1_5_6',150871,18,153331,214,155038,113,157154,707,0,0,0,0],['1_5_7',150924,211,153361,112,155064,207,157181,312,0,0,0,0],['1_5_9',169496,0,169497,0,155100,414,156421,12,0,0,0,0],['1_5_8',169506,0,153123,106,155332,213,156490,319,0,0,0,0],['1_6_2',151116,115,153442,304,155345,113,156446,12,0,0,0,0],['1_6_3',151305,107,153463,113,155413,14,156469,115,0,0,0,0],['1_6_4',151876,413,153690,216,155391,320,156522,718,0,0,0,0],['1_6_5',151874,310,153554,318,155470,214,157265,114,0,0,0,0],['1_6_6',151490,313,153599,114,155490,117,157288,1011,0,0,0,0],['1_6_7',151517,210,153643,309,155546,308,157307,209,0,0,0,0],['1_6_9',169498,0,169499,0,155592,8,157322,112,0,0,0,0],['1_6_8',169507,0,153743,104,156002,118,156531,306,0,0,0,0],['1_7_2',151116,115,153442,304,155345,113,156446,12,0,0,0,0],['1_7_3',151305,107,153463,113,155413,14,156469,115,0,0,0,0],['1_7_4',151858,209,153960,209,155508,306,156676,808,0,0,0,0],['1_7_5',151874,310,153554,318,155470,214,157265,114,0,0,0,0],['1_7_6',151490,313,153599,114,155490,117,157288,1011,0,0,0,0],['1_7_7',151517,210,153643,309,155546,308,157307,209,0,0,0,0],['1_7_9',169498,0,169499,0,155592,8,157322,112,0,0,0,0],['1_7_8',169507,0,153743,104,156002,118,156531,306,0,0,0,0],['1_8_8',151691,408,153987,311,156761,308,157447,220,0,0,0,0],['1_8_9',151704,18,152373,311,152374,521,152375,405,152379,313,0,0],['1_9_8',151691,408,153987,311,156761,308,157447,220,0,0,0,0],['1_9_9',151704,18,152373,311,152374,521,152375,405,152379,313,0,0],['1_16_3',153689,12,154301,10,154303,111,157602,908,0,0,0,0],['1_16_4',153731,20,153796,14,153798,409,153799,118,153800,17,0,0],['1_16_7',153777,110,156006,318,157374,407,0,0,0,0,0,0],['1_16_9',153824,13,156031,14,157384,208,0,0,0,0,0,0],['1_17_4',153731,20,153796,14,153798,409,153799,118,153800,17,0,0],['1_17_7',153777,110,156006,318,157374,407,0,0,0,0,0,0],['1_17_9',153824,13,156031,14,157384,208,0,0,0,0,0,0],['1_18_2',154030,8,154079,123,154409,407,154083,208,154411,118,154412,412],['1_18_3',154037,15,154414,101,154416,120,154417,2,154418,108,154419,303],['1_18_4',154075,307,156112,15,157868,312,0,0,0,0,0,0],['1_18_5',154090,14,156132,318,157479,716,0,0,0,0,0,0],['1_18_6',154455,106,156819,19,156659,412,0,0,0,0,0,0],['1_18_8',154151,113,156833,812,157526,218,0,0,0,0,0,0],['1_18_9',154473,8,156849,8,157545,918,0,0,0,0,0,0],['1_19_2',154030,8,154079,123,154409,407,154083,208,154411,118,154412,412],['1_19_3',154037,15,154414,101,154416,120,154417,2,154418,108,154419,303],['1_19_4',154075,307,156112,15,157868,312,0,0,0,0,0,0],['1_19_5',154090,14,156132,318,157479,716,0,0,0,0,0,0],['1_19_6',154455,106,156819,19,156659,412,0,0,0,0,0,0],['1_19_8',154151,113,156833,812,157526,218,0,0,0,0,0,0],['1_19_9',154473,8,156849,8,157545,918,0,0,0,0,0,0]]

In [ ]:
def lst_to_container_module(module_lst):

  stack_height=int((len(module_lst[0])-1)/2)

  # Sort the module based on cellKey, so we get all bays and stacks in right order
  module_lst = sorted(module_lst,key=lambda x: x[0])
  bays=dict()
  for index, stack in enumerate(module_lst):
    stack_lst=[]
    for i in range(1,stack_height+1):
      # Obtain the ID and time slot
      # ID at odd places, value at even place, it's always ID,value
      container_ID= stack[i*2-1]
      container_value= stack[i*2]
      # ID 0 -> No container
      if container_ID ==0:
        break
      else:
        # Container value 0 is a special container
        if container_value==0:
          stack_lst.append(Container(np.inf,container_ID))
        else:
          stack_lst.append(Container(container_value,container_ID))

    cellKey=stack[0]
    stack=Stack(stack_lst,stack_height,ID=cellKey)
    parts = cellKey.split("_")
    # Get bay and module index
    module_index=int(parts[0])
    bay_index=int(parts[1])

    # If bay already exists, add stack to it, otherwise create new bay
    bay=bays.get((module_index,bay_index),[])
    bay.append(stack)
    # Update bay with new stack added
    bays[module_index,bay_index]=bay
  bays = dict(sorted(bays.items()))
  # Returns a list of modules
  module=[Bay(bays[key],stack_height,ID=f"{key[0]}_{key[1]}") for key in bays]
  return Module(module,stack_height,ID=f"{module_index}")

In [ ]:
module=lst_to_container_module(module_lst)
print(module)

Bay 1_1:

   
216
319
206
inf
inf

Bay 1_2:

    |     |     |     |     |     |     |    
    |     |     | 406 |     |     | 322 |    
112 | 212 |  13 | 812 |  14 | 319 | 414 | 113
115 | 207 | 117 | 116 | 816 | 207 | 307 | 215
313 | 113 | 212 | 210 |  15 | 413 | 213 | inf
116 | 108 |  15 |  17 | 106 | 102 | 210 | inf

Bay 1_3:

    |     |     |     |     |    
    |     |     |     |     |    
112 | 212 |  13 |  14 | 319 | 113
115 | 207 | 117 | 816 | 207 | 215
313 | 113 | 212 |  15 | 413 | inf
116 | 108 |  15 | 106 | 102 | inf

Bay 1_4:

    |     |     |     |     |     |    
    |     |     |     |     |     |    
412 | 208 | 213 | 707 | 312 | 118 |  12
315 |  10 | 710 | 113 | 207 | 119 | 414
107 | 313 | 216 | 214 | 112 |   8 | inf
121 | 314 |  17 |  18 | 211 |  15 | inf

Bay 1_5:

    |     |     |     |     |     |    
    |     |     |     |     |     |    
412 | 208 | 213 | 707 | 312 | 319 |  12
315 |  10 | 710 | 113 | 207 | 213 | 414
107 | 313 | 216 | 214 | 112 | 106 | inf
12

In [ ]:
print(module.max_container())

99999
